In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
yesterday = "2018-09-17"

In [3]:
# Load from csv
df_product_hierachy = pd.read_csv('../dataset/product_hierarchy.csv', 
                                 delimiter= ',',
                                 dtype={'product_id':'category',
                                        'cluster_id':'category',
                                        'hierarchy1_id':'category',
                                        'hierarchy2_id':'category',
                                        'hierarchy3_id':'category',
                                        'hierarchy4_id':'category',
                                        'hierarchy5_id':'category'})
df_sales = pd.read_csv('../dataset/sales.csv',
                       delimiter=',', 
                       dtype={"product_id":"category", 
                              "store_id":"category",
                              "promo_type_1":"category",
                              "promo_bin_1":"category",
                              "promo_type_2":"category",
                              "promo_bin_2":"category",
                              "promo_discount_2":"category",
                              "promo_discount_type_2":"category"},
                       parse_dates=["date"])
df_sales = df_sales.join(df_product_hierachy.set_index('product_id'), on='product_id')

The Plan with this notebook was to check if there is any unexpected stock loss. Items which are not sold, but not in stock at the end of the day

In [4]:
df_sales_clean = df_sales
df_sales_clean["sales"] = df_sales_clean["sales"].fillna(0)
df_sales_clean["stock"] = df_sales_clean["stock"].fillna(0)
df_sales_clean = df_sales_clean.astype({"sales": int, "stock": int})

In [5]:
sales_stock = df_sales_clean.groupby(['store_id', 'date']).agg({'sales':'sum', 'stock':'sum'}).reset_index()

In [6]:
sales_stock['prev_stock'] = sales_stock.stock.shift(1)

In [7]:
sales_stock['stock_diff_after_sales'] = sales_stock['stock'] - (sales_stock['prev_stock'] - sales_stock['sales'])

In [8]:
sales_stock

,store_id,date,sales,stock,prev_stock,stock_diff_after_sales
0,S0001,2017-01-02,67.0,3280.0,NaN,NaN
1,S0001,2017-01-03,74.0,3255.0,3280.0,49.0
2,S0001,2017-01-04,99.0,3169.0,3255.0,13.0
3,S0001,2017-01-05,94.0,3192.0,3169.0,117.0
4,S0001,2017-01-06,50.0,3254.0,3192.0,112.0
...,...,...,...,...,...,...
157243,S0047,2019-12-25,0.0,0.0,0.0,0.0
157244,S0047,2019-12-26,0.0,0.0,0.0,0.0
157245,S0047,2019-12-27,0.0,0.0,0.0,0.0
157246,S0047,2019-12-28,0.0,0.0,0.0,0.0


Since there are shippments comming to each store each day, it is impossible to tell if items where stolen without having the shippment plan